<a href="https://colab.research.google.com/github/JLMuehlbauer/food-sales-prediction/blob/main/Project_1_food_sales_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Food Sales Predictions
---
#Project 1: Part 1
Jackson Muehlbauer

In [51]:
import pandas as pd

##Loading Data

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
#Reading data
filepath = '/content/drive/My Drive/Colab Notebooks/2: Week 2 Pandas/sales_predictions.csv'
df = pd.read_csv(filepath)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


##Data Cleaning

### 1. Number of (Rows, Columns)

In [55]:
# 1. Number of (Rows, Columns)
df.shape

(8523, 12)

### 2. What are the datatypes in each column?

In [56]:
# 2. What are the datatypes in each column?
df.dtypes 

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

### 3. Are there duplicated rows? If so, drop them

In [57]:
# 3. Are there duplicate rows? If so, drop them
df.duplicated().value_counts()

False    8523
dtype: int64

There are no duplicated rows

### 4. Are there missing values?

In [58]:
# 4. Are there missing values?
df.isna().sum()


Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [59]:
# Understanding Outlet_Size values
print(df['Outlet_Size'].value_counts())
print(f"NaN {df['Outlet_Size'].isna().sum()}")

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64
NaN 2410


### 5. Addressing missing values
---
Item_Weight and Outlet_Size both have missing values...

1. Item_Weight is of dtype float. I assume that it could be a useful column for predictions and less that 18% of the data is NaN, thus, I will remove the missing data from the dataframe. 

2. Outlet_Size is categorical data (with only 3 unique values) of type string. I also assume this could be a valuable column for predictions. There may also be some key learnings that can be made by keeping this data, thus, I will replace missing values with "missing"

In [60]:
# Dropping NaN Item_Weight
df.dropna(subset = ['Item_Weight'], inplace = True)

In [61]:
# Filling NaN values in Outlet_Size with 'missing'
df['Outlet_Size'].fillna('missing', inplace = True)


### 6. Confirm there are no missing values

In [62]:
# Confirming that there are no longer missing values
df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

### 7) Find and fix any inconsistent categories of data (example: fix cat, Cat, and cats so that they are consistent).


In [63]:
print(df.nunique())
print(df.dtypes)

Item_Identifier              1555
Item_Weight                   415
Item_Fat_Content                5
Item_Visibility              6519
Item_Type                      16
Item_MRP                     5225
Outlet_Identifier               8
Outlet_Establishment_Year       8
Outlet_Size                     4
Outlet_Location_Type            3
Outlet_Type                     3
Item_Outlet_Sales            3066
dtype: int64
Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object


In [64]:
#checking Item_Fat_Content for inconsistencies
df['Item_Fat_Content'].value_counts()

Low Fat    4222
Regular    2388
LF          260
reg         106
low fat      84
Name: Item_Fat_Content, dtype: int64

1. I assume that 'Low Fat', 'LF', and 'low fat' are all the same category. Thus, I wils change all values to Low Fat
2. I assume 'Regular' and 'reg' are the same, thus, I will change all values to 'Regular'

In [65]:
# making filters
low_fat = (df['Item_Fat_Content'] == 'LF') | (df['Item_Fat_Content'] == 'low fat')
reg_fat = df['Item_Fat_Content'] == 'reg'

In [66]:
# Using .loc to apply these filters to change the values
df.loc[low_fat, 'Item_Fat_Content'] = 'Low Fat'
df.loc[reg_fat, 'Item_Fat_Content'] = 'Regular'

#Checking that the changes were made
df['Item_Fat_Content'].value_counts()

Low Fat    4566
Regular    2494
Name: Item_Fat_Content, dtype: int64

In [67]:
#checking Item_Type for inconsistencies
print(df['Item_Type'].value_counts())
print('\n')
print(df['Outlet_Identifier'].value_counts())
print('\n')
print(df['Outlet_Size'].value_counts())
print('\n')
print(df['Outlet_Location_Type'].value_counts())
print('\n')
print(df['Outlet_Type'].value_counts())
#checks out

Fruits and Vegetables    1019
Snack Foods               988
Household                 759
Frozen Foods              718
Dairy                     566
Canned                    539
Baking Goods              536
Health and Hygiene        430
Soft Drinks               374
Meat                      337
Breads                    204
Hard Drinks               183
Others                    137
Starchy Foods             130
Breakfast                  89
Seafood                    51
Name: Item_Type, dtype: int64


OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
Name: Outlet_Identifier, dtype: int64


missing    2410
Small      1860
Medium     1858
High        932
Name: Outlet_Size, dtype: int64


Tier 2    2785
Tier 3    2415
Tier 1    1860
Name: Outlet_Location_Type, dtype: int64


Supermarket Type1    5577
Supermarket Type2     928
Grocery Store         555
Name: Outlet_Type, dtype: int64


### 8. For any numerical columns, obtain the summary statistics of each (min, max, mean)


In [68]:
# Summary statistics
df.describe(exclude=[object])

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,7060.000000,7060.000000,7060.000000,7060.000000
mean,12.857645,0.063963,141.240683,2000.490935,2118.626808
std,4.643456,0.048625,62.411888,6.588984,1533.445842
min,4.555000,0.000000,31.490000,1987.000000,33.290000
25%,8.773750,0.026768,94.143600,1997.000000,922.798800
50%,12.600000,0.052493,142.729900,2002.000000,1789.670400
75%,16.850000,0.092774,186.024000,2007.000000,2966.139000
max,21.350000,0.311090,266.888400,2009.000000,10256.649000


In [69]:
# Only showing min, max, mean
print(f'Minimum value: \n{df.min(numeric_only = True)}\n')
print(f'Minimum value: \n{df.max(numeric_only = True)}\n')
print(f'Minimum value: \n{df.mean(numeric_only = True)}\n')


Minimum value: 
Item_Weight                     4.555
Item_Visibility                 0.000
Item_MRP                       31.490
Outlet_Establishment_Year    1987.000
Item_Outlet_Sales              33.290
dtype: float64

Minimum value: 
Item_Weight                     21.35000
Item_Visibility                  0.31109
Item_MRP                       266.88840
Outlet_Establishment_Year     2009.00000
Item_Outlet_Sales            10256.64900
dtype: float64

Minimum value: 
Item_Weight                    12.857645
Item_Visibility                 0.063963
Item_MRP                      141.240683
Outlet_Establishment_Year    2000.490935
Item_Outlet_Sales            2118.626808
dtype: float64



##Exploratory Visuals

##Explanatory Visuals